In [35]:
REG_NUM_PAIR = {
    '101':'第M130020507号',
    '102':'第M130020508号',
    '201':'第M130020509号',
    '202':'第M130020510号',
    '第M130020507号':'101',
    '第M130020508号':'102',
    '第M130020509号':'201',
    '第M130020510号':'202',
}
LOGIN_URL = 'https://www.minpaku.mlit.go.jp/jigyo/login'
REPORT_URL_TEMPLATE = "https://www.minpaku.mlit.go.jp/jigyo/a05/e?retURL=%2Fjigyo%2Fapex%2Fjissekilist&scontrolCaching=1&year={year}&month={month}&TodokedeNumber={registeredNo}"

In [46]:
from selenium import webdriver
from selenium.webdriver.chrome.webdriver import WebDriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.alert import Alert
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from datetime import datetime
import os
import time
from dotenv import load_dotenv
load_dotenv()
USERNAME = os.environ['USERNAME']
PASSWORD = os.environ['PASSWORD']
RETRY_ATTEMPT = int(os.environ['RETRY_ATTEMPT'])

def construct_date(dt:datetime):
    month = f"{dt.month:02d}"
    # 年度は4月からなので、４より小さい月は、前年度になる
    year = dt.year-1 if dt.month < 4 else dt.year
    return (year, month)
PREV_REPORT_PERIOD = {
    # curernt_month: (prev_month, year_adjust)
    12:(10, 0),
    1:(10, -1),
    2:(12,-1),
    3:(12, -1),
    4:(2, 0),
    5:(2,0),
    6:(4, 0),
    7:(4,0),
    8:(6,0),
    9:(6,0),
    10:(8,0),
    11:(8,0)
}
def get_previous_report_period_start_datetime():
    current_year = datetime.now().year
    current_month = datetime.now().month
    return datetime(year=current_year+PREV_REPORT_PERIOD[current_month][1], month=PREV_REPORT_PERIOD[current_month][0], day=1)

def submit(driver:WebDriver, roomNo:str, retry_attempt:int=RETRY_ATTEMPT):
    if retry_attempt <= 0:
        return
    year_month = construct_date(get_previous_report_period_start_datetime())
    target_url = REPORT_URL_TEMPLATE.format_map({
        'year':year_month[0],
        'month':year_month[1],
        'registeredNo': REG_NUM_PAIR[roomNo]
        })
    try:
        wait = WebDriverWait(driver, RETRY_ATTEMPT)
        driver.get(target_url)
        save_button_locator = (By.ID, 'Jisseki_SaveButton')
        wait.until(EC.element_to_be_clickable(save_button_locator))
        driver.find_element(*save_button_locator).click()
        wait.until(EC.alert_is_present())
        Alert(driver).accept()
        close_button_locator = (By.CSS_SELECTOR, 'input[value="閉じる"]')
        wait.until(EC.element_to_be_clickable(close_button_locator))
        driver.find_element(*close_button_locator).click()
        time.sleep(1)
    except TimeoutException:
        print(f"Retrying... ({RETRY_ATTEMPT-retry_attempt-1}/{RETRY_ATTEMPT})")
        submit(driver, roomNo, retry_attempt-1)
        
class ChromeDriver(object):
    def __init__(self, excutable_path:str):
        options = Options()
        options.add_argument('--headless')
        service = Service(executable_path=excutable_path)
        self.driver = webdriver.Chrome(service=service, options=options)

    def __enter__(self):
        return self.driver

    def __exit__(self, exception_type, exception_value, traceback):
        self.driver.close()

In [48]:
with ChromeDriver(excutable_path='./webdrivers/chromedriver') as driver:
    driver.get(LOGIN_URL)
    wait = WebDriverWait(driver, 10)
    print("Loading login page...")
    username_elem_locator = (By.ID, "username")
    wait.until(EC.element_to_be_clickable(username_elem_locator))
    print("Login page loaded.")
    driver.find_element(*username_elem_locator).send_keys(USERNAME)

    password_elem_locator = (By.ID, "password")
    wait.until(EC.element_to_be_clickable(password_elem_locator))
    driver.find_element(*password_elem_locator).send_keys(PASSWORD)

    driver.find_element(by=By.ID, value="Login").click()
    print("Login...")
    # eait until logged in page show
    wait.until(EC.presence_of_element_located((By.ID, "networkNameHeaderText")))

    print("Started reporting")
    for roomNo in ['101', '102', '201', '202']:
        # submit(driver, roomNo, RETRY_ATTEMPT)
        print(roomNo)

Loading login page...
Login page loaded.
Login...
Started reporting
101
102
201
202
